In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from paperpi.library import Plugin, CacheFiles, get_help
from importlib import import_module
from pathlib import Path
import paperpi.my_constants as paperpi_constants
import logging
from IPython.display import Image 
import argparse
import sys
import re


In [4]:
logging.basicConfig(level='INFO')

In [5]:
logger = logging.getLogger(__name__)

In [58]:
def setup_plugins(project_root, plugin_list=None, resolution=(640, 400), skip_layouts=False):
    '''create dictionary of plugins using sample configurations provided in each plugin directory
    
    Args:
        project_root(`str`): directory where main script is located
        use_all_layouts(`Bool`): use all discovered layouts
        plugin_list(`list`): list of plugins to update; when None all discovered plugins will be updated
        resolution(`tuple` of `int`): screen resolution to use when generating images
        load_layouts(`bool`): when True, create a full Plugin object with a layout and an image
        
    Returns:
        `dict` of Plugin objects and layout name used to generate the Plugin
        '''


    def font_path(layout):
        '''add font path to layout'''
        for k, block in layout.items():
            font = block.get('font', None)
            if font:
                font = font.format(paperpi_constants.FONTS)
                block['font'] = font
        return layout
    
    plugin_path = Path(f'{project_root}/{paperpi_constants.PLUGINS}')
    
    # discover all plugins
    plugins = get_help._get_modules(plugin_path)
    logger.debug(f'discovered plugins: {plugins}')
    
    # reduce the list down to just the specified plugins
    if plugin_list:
        my_list = []
        for i in plugins:
            if i in plugin_list:
                my_list.append(i)
            else:
                print(f'specified plugin not found: {i}')
        plugins = my_list
    
    cache = CacheFiles()
    
    plugin_dict = {}
    
    
    # get the base name for the plugins package
    pkg_name = '.'.join([i for i in plugin_path.parts if not i.startswith('.')])

    for plugin in plugins:
        plugin_dict[plugin] = []
        all_layouts = {}
        print(f'found plugin {plugin}')
#         pkg_name = f'{".".join(plugin_path.parts)}'
        logger.debug(f'importing pkg: {pkg_name}')
        
        # import the plugin, layout and sample configuration
        try:
            module = import_module(f'{pkg_name}.{plugin}')            
            layout_import =  import_module(f'{pkg_name}.{plugin}.layout')
            sample_import = import_module(f'{pkg_name}.{plugin}.sample')
        except ModuleNotFoundError as e:
            print(f'skipping plugin {plugin} due to previous error: {e}')
            continue
        
        # get all of the layout dictionarys from the layout file
        for a in dir(layout_import):
            if not a.startswith('_') and isinstance(getattr(layout_import, a), dict):
                all_layouts[a] = (getattr(layout_import, a))
                
        if len(all_layouts) < 1:
            print(f'skipping plugin {plugin} due to missing layouts')
            continue
            
        # remove the default layout from the list if there are multiple layouts defined
#         if len(all_layouts) > 1 and 'layout' in all_layouts:
#             all_layouts.pop('layout')
        
        # make sure there is a valid configuration:
        try:
            config = sample_import.config
        except AttributeError as e:
            print(f'skipping plugin {plugin} due to missing sample configuration: {e}')
            continue
        
        # setup plugin
        if skip_layouts:
            print('skipping creating full layouts and creating images due to command line switch (-s)')
        for name, layout in all_layouts.items():
            print(f'adding layout: {name}')
            layout = font_path(layout)
            if not skip_layouts:
                my_plugin = Plugin(resolution=resolution,
                                   cache=cache,
                                   layout=layout,
                                   update_function=module.update_function,
                                   config=config
                                  )
                my_plugin.refresh_rate = 1

                try:
                    if 'kwargs' in config:
                        my_plugin.update(**config['kwargs'])

                    else:
                        my_plugin.update()
                except Exception as e:
                    print(f'plugin "{plugin}" could not be configured due to errors: {e}')
            else:
                my_plugin = None
            plugin_dict[plugin].append({
                                   'plugin': plugin,
                                   'module': module, 
                                   'plugin_obj': my_plugin, 
                                   'doc_path': plugin_path/plugin,
                                   'layout': name})
    return plugin_dict


In [59]:
# pd = setup_plugins('../paperpi')

DEBUG:__main__:discovered plugins: ['reddit_quote', 'librespot_client', 'moon_phase', 'lms_client', 'splash_screen', 'basic_clock', 'xkcd_comic', 'word_clock', 'demo_plugin', 'newyorker', 'default', 'met_no', 'crypto', 'dec_binary_clock']
DEBUG:__main__:importing pkg: paperpi.plugins
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [-------------text-------------]
DEBUG:root:section: [---------attribution----------]
DEBUG:root:section: [-------------time-------------]
DEBUG:root:[[....calculating layouts....]]
INFO:root:section: [.............text.............]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, height: 0.65
DEBUG:root:scaling font size
DEBUG:root:y target size reached
DEBUG:root:calculated font size: 37
DEBUG:root:absolute coordinates provided
DEBUG:root:block coordinates: (0, 0)
INFO:root:section: [.........attribution..........]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, height: 0.25
DEBUG:root:scaling font size
DEBUG:root:

found plugin reddit_quote
adding layout: layout


DEBUG:root:cached data expired, fetching fresh data
DEBUG:root:downloading fresh data from reddit
DEBUG:root:fetching data from reddit
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.reddit.com:443
DEBUG:urllib3.connectionpool:https://www.reddit.com:443 "GET /r/quotes/top/.json?t=week&limit=100 HTTP/1.1" 200 23368
DEBUG:root:processing 100 quotes
INFO:root:caching data
DEBUG:root:choosing quote with length < 144 characters
DEBUG:root:{'len': 52, 'text': 'The same flame that melts the butter hardens the egg', 'attribution': None}
DEBUG:root:"len" is not a recognized block, skipping
DEBUG:root:formatting string: The same flame that melts the butter hardens the egg
DEBUG:root:text size: (477, 71)
DEBUG:root:pasting hcenterd
DEBUG:root:pasting vcentered
DEBUG:root:paste coordinates: (81, 94)
DEBUG:root:formatting string: 21:20
DEBUG:root:text size: (88, 39)
DEBUG:root:paste coordinates: (0, 0)
DEBUG:root:"tag_image" is not a recognized block, skipping
DEBUG:root:[[----c

adding layout: quote


DEBUG:root:using cached reddit data
DEBUG:root:choosing quote with length < 144 characters
DEBUG:root:{'len': 28, 'text': "I ain't got time to bleed", 'attribution': '―?'}
DEBUG:root:"len" is not a recognized block, skipping
DEBUG:root:formatting string: I ain't got time to bleed
DEBUG:root:text size: (367, 39)
DEBUG:root:pasting hcenterd
DEBUG:root:pasting vcentered
DEBUG:root:paste coordinates: (136, 110)
DEBUG:root:formatting string: ―?
DEBUG:root:text size: (23, 31)
DEBUG:root:pasting hcenterd
DEBUG:root:pasting vcentered
DEBUG:root:paste coordinates: (308, 34)
DEBUG:root:adding border to image
DEBUG:root:adding border to side top
DEBUG:root:formatting string: 21:20
DEBUG:root:text size: (88, 39)
DEBUG:root:paste coordinates: (0, 0)
DEBUG:root:"tag_image" is not a recognized block, skipping
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [-------------text-------------]
DEBUG:root:section: [---------attribution----------]
DEBUG:root:section: [----------

...

INFO:root:the text will spill outside of padded area using these values
DEBUG:root:pasting vcentered
DEBUG:root:paste coordinates: (0, -1)
INFO:root:section: [__________sparkline___________]
DEBUG:root:setting block type: ImageBlock
DEBUG:root:block area: (640, 220)
DEBUG:root:padded area: [640, 220]
DEBUG:root:fill: 0, bkground: 255
DEBUG:root:creating Block
DEBUG:root:image area (max): (640, 220)
DEBUG:root:no image set; setting to blank image with area: (640, 220)
INFO:root:section: [_________update_time__________]
DEBUG:root:setting block type: TextBlock
DEBUG:root:block area: (640, 60)
DEBUG:root:padded area: [636, 56]
DEBUG:root:fill: 0, bkground: 255
DEBUG:root:creating Block
DEBUG:root:setting old_font = None
DEBUG:root:calculating maximum characters for font ('Josefin Sans', 'Light')
DEBUG:root:average character width: 16.02173913043478
DEBUG:root:maximum characters per line: 40
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (92, 30)
DEBUG:root:pasting hcenterd
DEBUG

found plugin dec_binary_clock
adding layout: dec_binary_clock


DEBUG:root:maximum characters per line: 53
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (50, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (478, 7)
DEBUG:root:TIME = None
DEBUG:root:image area (max): (640, 356)
DEBUG:root:image dimensions: (760, 690)
DEBUG:root:idx:0 diemension (760) is greater than idx:0 padded_area: 630
DEBUG:root:idx:1 diemension (690) is greater than idx:1 padded_area: 346
DEBUG:root:resizing image to: [630, 346]
DEBUG:root:padding: 5
DEBUG:root:h centering: x=129
DEBUG:root:v centering: y=5
DEBUG:root:pasting image at: 129, 5
DEBUG:root:formatting string: 21:22
DEBUG:root:text size: (54, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (141, 7)
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [-----------bin_img------------]
DEBUG:root:section: [-------------time-------------]
DEBUG:root:[[....calculating layouts....]]
INFO:root:section: [...........bin_img........

adding layout: layout


DEBUG:root:maximum characters per line: 53
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (50, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (411, 6)
DEBUG:root:TIME = None
DEBUG:root:image area (max): (640, 356)
DEBUG:root:image dimensions: (760, 690)
DEBUG:root:idx:0 diemension (760) is greater than idx:0 padded_area: 630
DEBUG:root:idx:1 diemension (690) is greater than idx:1 padded_area: 346
DEBUG:root:resizing image to: [630, 346]
DEBUG:root:padding: 5
DEBUG:root:h centering: x=129
DEBUG:root:v centering: y=5
DEBUG:root:pasting image at: 129, 5
DEBUG:root:formatting string: 21:22
DEBUG:root:text size: (54, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (196, 5)


In [9]:
def update_ini_file(plugin_dict):
    '''append sample configurations for each module to the default paperpi.ini file
    distributed at install'''
    
    base_ini_file = '../install/paperpi_base.ini'
    output_ini_file = '../paperpi/config/paperpi.ini'
    
    config_sections = []
    
    print(f'updating {output_ini_file} using sample configs from plugins')
    
    for plugin, data in plugin_dict.items():
        try:
            sample_config = data[0]['module'].constants.sample_config
        except (IndexError, AttributeError):
            logger.info(f'no valid data for plugin {plugin}')
            continue
            
        # check that it matches the format
        match = re.match('^\s{0,}\[Plugin', sample_config)
        try:
            if match.string:
                sample_config = re.sub('^\s{0,}\[Plugin', '[xPlugin', sample_config)
            else:
                print(f'plugin {plugin} does not have a standard sample_config string. Please check formatting.')
                continue
        except AttributeError:
            print(f'plugin {plugin} has no valid sample_config string: {sample_config}')
            continue
        
        config_sections.append(sample_config)
        config_sections.append('\n')
    
    output_ini_list = []
    with open(base_ini_file, 'r') as base_f:
        for i in base_f:
            output_ini_list.append(i)
    
    output_ini_list.extend(config_sections)
    
    with open(output_ini_file, 'w') as out_f:
        for i in output_ini_list:
            out_f.write(i)

In [60]:
def create_readme(plugin_dict, project_root, overwrite_images=False):
    plugin_docs = {}
    plugin_path = Path(f'{project_root}/{paperpi_constants.PLUGINS}')
    readme_name = 'README'
    readme_additional = '_additional'
    readme_suffix = '.md'
    
    # run through each plugin and gather the document information
    for plugin, layouts in plugin_dict.items():
        print(f'Processing plugins: {plugin}')
        try:
            doc_path = Path(layouts[0]['doc_path'])
            plugin_readme = Path(doc_path)/f'{readme_name}{readme_suffix}'
            plugin_additional_readme = Path(doc_path)/f'{readme_name}{readme_additional}{readme_suffix}'
        except IndexError:
            print(f'   skipping {plugin} due to incomplete documentation')
            continue
        
        # pull the help information for the plugin
        readme_text = get_help.get_help(plugin, False, plugin_path=plugin_path)
        
        # read the additional documentation
        if plugin_additional_readme.exists():
            with open(plugin_additional_readme, 'r') as file:
                additional_text = file.read()
        else:
            additional_text = '' 
        
        default_layout_image = {'filename': 'not found',
                                'path': 'none',
                                'layout_name': 'layout'}
        layout_images = []
        # gather all the images associated with the plugin
        for layout in layouts:
            layout_name = layout['layout']
            image_filename = f'{plugin}.{layout["layout"]}-sample.png'
            image_path = Path(doc_path)/image_filename
                        
            print(f'   processing layout: {layout_name}')
            try:
                image = layout['plugin_obj'].image
            except AttributeError:
                print('  no image available, skipping this layout')
                continue
                
            if (image_path.exists() and overwrite_images) or not image_path.exists():
                print(f'   saving image: {image_path}')
                image.save(image_path)
            else:
                print(f'   image exists, skipping save')
                
            
            
            layout_entry = {'filename': image_filename,
                            'path': image_path,
                            'layout_name': layout_name}
            
            # try to locate the default "layout" to show at the top of the documentation
            if layout_name == 'layout':
                default_layout_image.update(layout_entry)
            
            layout_images.append(layout_entry)

        print('   writing README files')
        with open(plugin_readme, 'w') as file:
            plugin_name = ".".join(doc_path.parts)
            file.write(f'# {plugin}\n')
            file.write(f'![sample image for plugin {plugin_name}](./{default_layout_image["filename"]}) \n\n')
            file.write(f'```\n{readme_text}\n```\n\n')
            file.write(f'## Provided Layouts:\n\n')
            for l in layout_images:
                file.write(f'layout: **{l["layout_name"]}**\n\n')  
                file.write(f'![sample image for plugin {l["layout_name"]}](./{l["filename"]}) \n\n\n')

            file.write(additional_text)
        
        plugin_docs[plugin] = {'readme': plugin_readme, 'image': default_layout_image["path"]}
            
    
    return plugin_docs

In [61]:
# plugin_d = create_readme(pd, '../paperpi/', False)

DEBUG:paperpi.library.get_help:gathering information for: reddit_quote
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.reddit_quote.reddit_quote
DEBUG:paperpi.library.get_help:gathering information for: librespot_client
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.librespot_client.librespot_client
DEBUG:paperpi.library.get_help:gathering information for: moon_phase
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.moon_phase.moon_phase
DEBUG:paperpi.library.get_help:gathering information for: lms_client
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.lms_client.lms_client
DEBUG:paperpi.library.get_help:gathering information for: splash_screen
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.splash_screen.splash_screen
DEBUG:paperpi.library.get_help:gathering information for: basic_clock
DEBUG:paperpi.library.get_help:attempting to import: paperpi.plugins.basic_clock.basic_clock
DEBU

Processing plugins: reddit_quote
   processing layout: layout
   image exists, skipping save
   processing layout: quote
   image exists, skipping save
   processing layout: quote_inverse
   image exists, skipping save
   processing layout: quote_small_screen
   image exists, skipping save
   writing README files
Processing plugins: librespot_client
   processing layout: layout
   image exists, skipping save
   processing layout: three_rows_text_only
   image exists, skipping save
   processing layout: two_column_three_row
   image exists, skipping save
   processing layout: two_rows_text_only
   image exists, skipping save
   writing README files
Processing plugins: moon_phase
   processing layout: layout
   image exists, skipping save
   processing layout: moon_data
   image exists, skipping save
   processing layout: moon_only
   image exists, skipping save
   writing README files
Processing plugins: lms_client
   processing layout: layout
   image exists, skipping save
   processin

In [11]:
def update_plugin_docs(plugin_docs, doc_path):
    '''update Plugin.md documentation with snips from all plugin READMEs
    
    Args:
        plugin_docs(dict): dictionary provided by create_readme'''
    doc_path = Path(doc_path)
    plugin_readme_source = Path(doc_path/'source/Plugins.md')
    plugin_readme_post_source = Path(doc_path/'source/Plugins_post.md')
    plugin_readme_final = Path(doc_path/plugin_readme_source.name)
    print('updating Plugins.md...')
    print(f'using: {plugin_readme_source}')
    print(f'using postscript file: {plugin_readme_post_source}')
    with open(plugin_readme_source, 'r') as file:
        source = file.read()
        
    with open(plugin_readme_post_source, 'r') as file:
        post = file.read()
    
    with open(plugin_readme_final, 'w') as file:
        file.write(source)
        for plugin, values in plugin_docs.items():
            file.write(f'### [{plugin}]({Path("..")/values["readme"]})\n')
            file.write(f'![{plugin} sample Image]({Path("..")/values["image"]})\n\n')
            
        file.write(post)        

In [62]:
# update_plugin_docs(plugin_d, '../documentation')

updating Plugins.md...
using: ../documentation/source/Plugins.md
using postscript file: ../documentation/source/Plugins_post.md


In [12]:
def main():
    parser = argparse.ArgumentParser(description='create_docs')
 
    parser.add_argument('-o', '--overwrite_images', default=False, action='store_true',
                       help='overwrite existing images for plugins when updating README files')
    
    parser.add_argument('-p', '--plugin_list', default=None, nargs='*', 
                       help='list of specific plugins to process')
    
    parser.add_argument('-r', '--project_root', default='../paperpi/', nargs=1,
                       help='path to project root')
    
    parser.add_argument('-d', '--documentation_path', default='../documentation',
                       help='path to documentation directory')
    
# this breaks README creation
#     parser.add_argument('-s', '--skip_layouts', default=False, action='store_true',
#                         help='skip loading layouts and do not create images')
    
    
    args = parser.parse_args()
    
#     args = parser.parse_known_args()
    plugin_dict = setup_plugins(args.project_root, args.plugin_list)
    plugin_docs = create_readme(plugin_dict, 
                                project_root=args.project_root,
                                overwrite_images=args.overwrite_images,
                                )
    update_ini_file(plugin_dict)
    update_plugin_docs(plugin_docs, doc_path=args.documentation_path)
    
    

In [13]:
if __name__ == "__main__":
    if '-f' in sys.argv:
        logger.debug('looks like this is running in a Jupyter notebook')
        idx = sys.argv.index('-f')
        del sys.argv[idx:idx+2]    
    print('updating documents...')
    r = main()

DEBUG:__main__:looks like this is running in a Jupyter notebook
DEBUG:__main__:importing pkg: ...paperpi.plugins


updating documents...
found up plugin reddit_quote


TypeError: the 'package' argument is required to perform a relative import for '...paperpi.plugins.reddit_quote'

In [66]:
!jupyter-nbconvert --to python --template python_clean create_docs.ipynb

[NbConvertApp] Converting notebook create_docs.ipynb to python
[NbConvertApp] Writing 13008 bytes to create_docs.py
